In [6]:
import sys
sys.path.append('/your/path/to/pysatl-mpest')

['/usr/lib64/python313.zip', '/usr/lib64/python3.13', '/usr/lib64/python3.13/lib-dynload', '', '/home/iraedeus/Projects/pysatl-mpest/.venv/lib64/python3.13/site-packages', '/home/iraedeus/Projects/pysatl-mpest/.venv/lib/python3.13/site-packages', '/your/path/to/pysatl-mpest', '/your/path/to/pysatl-mpest', '/your/path/to/pysatl-mpest', '/home/iraedeus/Projects/pysatl-mpest']


###Stage 1 of experiment:
Creating dataset

Importing

In [7]:
from pathlib import Path

import numpy as np

from experimental_env.preparation.dataset_generator import (
    ConcreteDatasetGenerator,
    RandomDatasetGenerator,
)
from mpest.models import ExponentialModel, GaussianModel, WeibullModelExp

Setting up paths to working directory, sample size and seed

In [9]:
WORKING_DIR = Path("/your/path/to/experiment/stage_1")
SAMPLES_SIZE = 100

seed = 42

Creating random dataset generator and selecting mixtures for experiment.
Performing dataset generation.

In [10]:
r_generator = RandomDatasetGenerator(seed)
mixtures = [
    [ExponentialModel],
    [GaussianModel],
    [WeibullModelExp],
    [WeibullModelExp, GaussianModel],
    [ExponentialModel, GaussianModel],
    [WeibullModelExp, WeibullModelExp],
    [ExponentialModel, ExponentialModel],
]
for models in mixtures:
    r_generator.generate(SAMPLES_SIZE, models, Path(WORKING_DIR), exp_count=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.44it/s]


Initializing specific generators and adding the distributions we need and their parameters to the mixture that we will generate.

In [11]:
c_generator1 = ConcreteDatasetGenerator(seed)
models = [ExponentialModel]
c_generator1.add_distribution(models[0], [1.0], 1.0)
c_generator1.generate(SAMPLES_SIZE, Path(WORKING_DIR), 5)

c_generator2 = ConcreteDatasetGenerator(seed)
models = [GaussianModel]
c_generator2.add_distribution(models[0], [0, 1.0], 1.0)
c_generator2.generate(SAMPLES_SIZE, Path(WORKING_DIR), 5)

c_generator3 = ConcreteDatasetGenerator(seed)
models = [WeibullModelExp]
c_generator3.add_distribution(models[0], [1.0, 1.0], 1.0)
c_generator3.generate(SAMPLES_SIZE, Path(WORKING_DIR), 5)

c_generator4 = ConcreteDatasetGenerator(seed)
models = [WeibullModelExp]
c_generator4.add_distribution(models[0], [1.0, 1.0], 1.0)
c_generator4.generate(SAMPLES_SIZE, Path(WORKING_DIR), 5)

c_generator5 = ConcreteDatasetGenerator(seed)
models = [WeibullModelExp]
c_generator5.add_distribution(models[0], [1.0, 0.5], 1.0)
c_generator5.generate(SAMPLES_SIZE, Path(WORKING_DIR), 5)

c_generator6 = ConcreteDatasetGenerator(seed)
models = [GaussianModel, GaussianModel]
c_generator6.add_distribution(models[0], [-1.0, 2.5], 0.3)
c_generator6.add_distribution(models[1], [1.0, 0.5], 0.7)
c_generator6.generate(SAMPLES_SIZE, Path(WORKING_DIR), 10)

c_generator7 = ConcreteDatasetGenerator(seed)
models = [GaussianModel, GaussianModel]
c_generator7.add_distribution(models[0], [0.0, 1.5], 0.6)
c_generator7.add_distribution(models[1], [1.0, 1.0], 0.4)
c_generator7.generate(SAMPLES_SIZE, Path(WORKING_DIR), 10)

###Stage 2 of experiment:
Performing parameter estimation

Importing

In [12]:
import random
from pathlib import Path

from experimental_env.experiment.estimators import (
    LikelihoodEstimator,
    LMomentsEstimator,
)
from experimental_env.experiment.experiment_executors.random_executor import (
    RandomExperimentExecutor,
)
from experimental_env.preparation.dataset_parser import SamplesDatasetParser
from mpest.em.breakpointers import StepCountBreakpointer
from mpest.em.distribution_checkers import (
    FiniteChecker,
    PriorProbabilityThresholdChecker,
)

Setting up source path, where the dataset is located, working path and seed.

In [13]:
SOURCE_DIR = Path("/your/path/to/experiment/stage_1")
WORKING_DIR = Path("/your/path/to/experiment/stage_2")

random.seed(42)

Parse dataset, creating experiment executors with chosen methods and executing.

In [14]:
# Parse stage 1
parser = SamplesDatasetParser()
datasets = parser.parse(SOURCE_DIR)

# Execute stage 2
executor = RandomExperimentExecutor(WORKING_DIR, 5, seed)
executor.execute(
    datasets,
    LMomentsEstimator(
        StepCountBreakpointer(max_step=16),
        FiniteChecker() + PriorProbabilityThresholdChecker(),
    ),
)

executor = RandomExperimentExecutor(WORKING_DIR, 5, seed)
executor.execute(
    datasets,
    LikelihoodEstimator(
        StepCountBreakpointer(max_step=16),
        FiniteChecker() + PriorProbabilityThresholdChecker(),
    ),
)

Starting L-moments estimation


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.86it/s]


Starting L-moments estimation


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.74it/s]


Starting L-moments estimation


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.20it/s]


Starting Likelihood estimation


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.87it/s]


Starting Likelihood estimation


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.30s/it]


###Stage 3:
Analysis results of the stage 2

Importing

In [15]:
from pathlib import Path

from experimental_env.analysis.analysis import Analysis
from experimental_env.analysis.analyze_strategies.density_plot import DensityPlot
from experimental_env.analysis.analyze_strategies.error_convergence import (
    ErrorConvergence,
)
from experimental_env.analysis.analyze_strategies.time_plot import TimePlot
from experimental_env.analysis.analyze_summarizers.error_summarizer import (
    ErrorSummarizer,
)
from experimental_env.analysis.analyze_summarizers.time_summarizer import TimeSummarizer
from experimental_env.analysis.metrics import SquaredError
from experimental_env.experiment.experiment_parser import ExperimentParser


Setting up all paths

In [16]:
WORKING_DIR = Path("/your/path/to/experiment/stage_3")
LMOMENTS_DIR = Path("/your/path/to/experiment/stage_2/LM-EM")
LIKELIHOOD_DIR = Path("/your/path/to/experiment/stage_2/MLE-EM")

Setting up aspects in which we will analyze

In [17]:
results_1 = ExperimentParser().parse(LMOMENTS_DIR)
results_2 = ExperimentParser().parse(LIKELIHOOD_DIR)

analyze_actions = [DensityPlot(), TimePlot(), ErrorConvergence(SquaredError())]
analyze_summarizers = [ErrorSummarizer(SquaredError()), TimeSummarizer()]

Analysis

In [18]:
Analysis(WORKING_DIR, analyze_actions, analyze_summarizers).analyze(results_1, "LM-EM")
Analysis(WORKING_DIR, analyze_actions, analyze_summarizers).analyze(results_2, "MLE-EM")
Analysis(WORKING_DIR, analyze_actions, analyze_summarizers).compare(
    results_1, results_2, "LM-EM", "MLE-EM"
)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.46it/s]
/home/iraedeus/Projects/pysatl-mpest/mpest/models/weibull.py:76: RuntimeWarning: overflow encountered in exp
  return (ek / el) * (xl ** (ek - 1.0)) / np.exp(xl**ek)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.48it/s]
